In [1]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, BlockNumber

context, model_loader = create_cmf(show=True)

Credmark context created with 
- chain_id=1
- block_number=None
- chain_provider_url=http://192...8545
- model_loader_path=['/home/yangye/dev/credmark/credmark-models-py/models']
- api_url=https://gateway.credmark.com
- use_local_models=None



## 5. Translate above code into a model to return Records for Uniswap V2 pool's price for a block range


In [3]:
# Let's create a model for a range of block numbers
# It support cases when there are > 5000 Sync events in the range, or no event during the

import pandas as pd

from credmark.cmf.model import Model
from credmark.cmf.types import Records
from credmark.dto import DTOField


class ContractWithRange(Contract):
    block_range: int = DTOField(ge=0, description='Block range to query')


@Model.describe(
    slug='jit.uniswap-v2-reserve-range',
    version='1.1',
    input=ContractWithRange,
    output=Records)
class UniswapV2ReserveRange(Model):
    def run(self, input):
        current_reserve = self.context.run_model('jit.uniswap-v2-reserve',
                                                 input)

        token0 = Token(input.functions.token0().call())
        token1 = Token(input.functions.token1().call())

        dfs = []
        with input.ledger.events.Sync as q:
            while True:
                df = q.select(q.columns,
                              where=q.BLOCK_NUMBER.ge(context.block_number-input.block_range),
                              order_by=q.BLOCK_NUMBER.comma_(q.LOG_INDEX)).to_dataframe()

                if not df.empty:
                    dfs.append(df)

                if df.shape[0] < 5000:
                    break

        df_post_events = (pd.DataFrame(input.fetch_events(
            input.events.Sync,
            from_block=(
                (int(df.loc[df.shape[0]-1, 'block_number']) + 1)
                if not df.empty
                else self.context.block_number-input.block_range),
            to_block=self.context.block_number)))

        if not df_post_events.empty:
            df_post_events = (df_post_events
                              .loc[:, ['blockNumber', 'reserve0', 'reserve1']]
                              .rename(columns={'blockNumber': 'block_number'}))

        if len(dfs) == 0:
            df_clean = pd.DataFrame()
        else:
            df_clean = (pd.concat(dfs)
                        .drop_duplicates(subset=['block_number'], keep='last')
                        .loc[:, ['block_number', 'evt_reserve0', 'evt_reserve1']]
                        .rename(columns={'evt_reserve0': 'reserve0', 'evt_reserve1': 'reserve1'})
                        .assign(block_number=lambda x: x.block_number.astype(int),
                                reserve0=lambda x: x.reserve0.apply(int),
                                reserve1=lambda x: x.reserve1.apply(int),))

        df_clean = pd.concat([df_clean, df_post_events]).reset_index(drop=True)

        if df_clean.empty or df_clean.loc[df_clean.shape[0]-1, 'block_number'] != context.block_number:
            df_clean = pd.concat([
                df_clean,
                pd.DataFrame({'block_number': [self.context.block_number],
                              'reserve0': [current_reserve['reserve0']],
                              'reserve1': [current_reserve['reserve1']]})])

        df_clean['price_token1'] = token0.scaled(df_clean['reserve0']) / token1.scaled(df_clean['reserve1'])
        df_clean['price_token0'] = 1 / df_clean['price_token1']

        df_clean = (df_clean
                    .merge(pd.DataFrame({'block_number': range(context.block_number-1000, context.block_number+1)}), how='right')
                    .sort_values('block_number')
                    .fillna(method='ffill')
                    .fillna(method='bfill'))

        return Records.from_dataframe(df_clean)


context.add_model(UniswapV2ReserveRange)

In [4]:
context.reset_cache()
df_range = context.run_model('jit.uniswap-v2-reserve-range',
                             {'address': '0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc', 'block_range': 1000},
                             return_type=Records).to_dataframe()

df_range.plot('block_number', 'price_token1')

ModelNotFoundError: Model jit.uniswap-v2-reserve (any version) not found